pure title tekst matching

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np #
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        if filename=="y_train.npy":
            print('reading train')            
            y_train=np.read_csv(os.path.join(dirname, filename) )#,error_bad_lines=False)#,names=['id','body','headline']
                    
        if filename.find("rain")>0:
            print('reading train')            
            train=pd.read_csv(os.path.join(dirname, filename) )#,error_bad_lines=False)#,names=['id','body','headline']
        if filename.find("est")>0:
            print('reading test')
            test=pd.read_csv(os.path.join(dirname, filename) )#,error_bad_lines=False )



In [ ]:
train

In [ ]:
test

In [ ]:
total=train.append(test,ignore_index=True)
total

In [ ]:
def bestsimilar(dtrain,tekst,label,indexv):
    if True:
        ncomps=300
        from sklearn.decomposition import TruncatedSVD
        tsvd=TruncatedSVD(n_components=ncomps, n_iter=7, random_state=42)
        from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
        from sklearn.datasets import fetch_20newsgroups
        news = fetch_20newsgroups(subset='train')
        print('#vectorizing text')
        kolom=dtrain.describe().T

        vectc=TfidfVectorizer(ngram_range=(2,3),analyzer='char_wb')
        vectw=TfidfVectorizer(ngram_range=(1,2))
        
        dtrain['txt']=dtrain[tekst]+' '+dtrain['image_phash']
        dfidf=vectc.fit_transform(dtrain['txt'])
        dfidfw=vectw.fit_transform(dtrain['txt'])

        print(dfidf.shape,dfidfw.shape)
        dfidf=tsvd.fit_transform(dfidf)
        dfidfw=tsvd.fit_transform(dfidfw)
        
        print('svd')

        
        from sklearn.metrics.pairwise import cosine_similarity,laplacian_kernel,manhattan_distances
        print('cosim')
        matchdb=cosine_similarity(np.hstack((dfidf, dfidfw)),np.hstack((dfidf[-3:], dfidfw[-3:])))
        for xi in range(3):
            dtrain['svd'+str(xi)]=matchdb[:,xi]
                
    return dtrain
matchdb=bestsimilar(total,'title','label_group','posting_id') 

find matching products sort on cosinus similarity 

In [ ]:
matchdb.sort_values('svd0')[-20:]

display test images

In [ ]:
BASE = '../input/shopee-product-matching/train_images/'
import cv2, matplotlib.pyplot as plt
pad = '../input/shopee-product-matching/test_images/'

def displayDF(train, random=False, COLS=5, ROWS=10, path=BASE):
    for k in range(ROWS):
        plt.figure(figsize=(20,5))
        for j in range(COLS):
            if random: row = np.random.randint(0,len(train))
            else: row = COLS*k + j
            name = train.iloc[row,1]
            title = train.iloc[row,3]
            title_with_return = ""
            for i,ch in enumerate(title):
                title_with_return += ch
                if (i!=0)&(i%20==0): title_with_return += '\n'
            #print(path,name)
            img = cv2.imread(path+name)
            plt.subplot(1,COLS,j+1)
            plt.title(title_with_return)
            plt.axis('off')
            plt.imshow(img)
        plt.show()


displayDF(test,random=False,COLS=3,ROWS=1,path=pad)

first matchings images

In [ ]:

BASE = '../input/shopee-product-matching/train_images/'

displayDF(matchdb.sort_values('svd0')[-51:-1],random=False)

second matching images

In [ ]:
displayDF(matchdb.sort_values('svd1')[-55:-1],random=False)

third matching images

In [ ]:
displayDF(matchdb.sort_values('svd2')[-55:-1],random=False)